# Homework Problem: Archaeological Seismic Tomography

## Problem Statement

A 20m × 20m concrete slab has a famous IU artifact hidden inside. It is too risky to tear up the slab to find the artifact. But geophysics comes to the rescue, and we will use **seismic tomography**! 

### Setup
- The slab is divided into 20×20 square cells (1m × 1m each)
- Four different sets of seismic scans were conducted:
  1. **Horizontal scans** (20 ray paths, left to right)
  2. **Vertical scans** (20 ray paths, top to bottom) 
  3. **Diagonal scans** (39 ray paths, upper-left to lower-right)
  4. **Anti-diagonal scans** (39 ray paths, upper-right to lower-left)
- Travel time data (in seconds) are provided in text files for each scan type
- The artifact creates a **slowness anomaly** (slower seismic velocities)

### Physical Model
The forward model relates slowness to travel times:
$$t_i = \sum_{j=1}^{400} G_{ij} s_j$$

Where:
- $t_i$ = travel time for ray $i$ (seconds)
- $s_j$ = slowness of cell $j$ (s/m) 
- $G_{ij}$ = path length of ray $i$ through cell $j$ (meters)
- 400 = total number of cells (20×20)

## Tasks

### Part 1: Forward Model Construction (25 points)
1. **Create the geometry matrix $\mathbf{G}$** for all four scan types:
   - Use the proved make_scanG function
   

### Part 2: SVD Analysis (25 points)
1. **Perform SVD** on the combined geometry matrix $\mathbf{G}$:
   $$\mathbf{G} = \mathbf{U}\mathbf{S}\mathbf{V}^T$$

2. **Analyze the singular value spectrum**:
   - Plot singular values on a log scale
   - Calculate the condition number: $\kappa = s_1/s_n$
   - Plot cumulative energy: $E_p = \frac{\sum_{i=1}^p s_i^2}{\sum_{i=1}^{rank} s_i^2} \times 100\%$

3. **Determine effective rank**: How many singular values contain 90%, 95%, and 99% of the energy?

### Part 3: Truncated Pseudoinverse Inversion (35 points)
1. **Load the travel time data** from the provided text files:
   - `horizontal_times.txt`
   - `vertical_times.txt`  
   - `diagonal_times.txt`
   - `antidiagonal_times.txt`

2. **Compute truncated pseudoinverse solutions** for different truncation levels:
   $$\mathbf{s}_p = \mathbf{V}_p \mathbf{S}_p^{-1} \mathbf{U}_p^T \mathbf{t}$$
   
   Test truncation values: $p = [10, 20, 30, 50, 75, 90, 100, 118]$

3. **For each solution**:
   - Reshape the 400×1 slowness vector into a 20×20 image
   - Plot the recovered slowness image
   - Calculate data fit: $\chi^2 = \|\mathbf{t} - \mathbf{G}\mathbf{s}_p\|^2$

### Part 4: Analysis and Interpretation (15 points)

1. **Determine optimal truncation**:
   - What is the **minimum number of singular values** $p$ needed to recover a recognizable artifact image?
   - Justify your choice based on:
     - Visual quality of the reconstruction
     - Data fit metrics
     - Stability of the solution

2. **Physical interpretation**:
   - Estimate the artifact's location, size, and shape. What is this symbol?
   - What is the slowness contrast of the artifact?
   - Discuss resolution limitations and trade-offs



In [21]:

# --- Helper function to build G matrices - MATLAB equivalent implementation ---
def make_scanG(numrows, numcols, cell_length):
    """
    Construct G matrix for 2D tomography problem. 
    Direct translation of MATLAB code with careful indexing conversion.
    """
    import numpy as np
    
    N = numrows * numcols
    
    # Initialize matrices - match MATLAB exactly
    Ghz = np.zeros((numrows, N))
    Gvert = np.zeros((numcols, N))
    Gdiag1 = np.zeros((numcols, N))
    Gdiag2 = np.zeros((numcols, N))
    Gdiag3 = np.zeros((numcols, N))
    Gdiag4 = np.zeros((numcols, N))
    
    # Vertical scans - MATLAB: for k=1:numcols, Gvert(k,(k-1)*numrows+1:k*numrows)=1; end
    for k in range(1, numcols + 1):
        start_matlab = (k-1) * numrows + 1  # MATLAB 1-based
        end_matlab = k * numrows            # MATLAB inclusive
        start_py = start_matlab - 1         # Convert to 0-based
        end_py = end_matlab                 # Python range is exclusive
        Gvert[k-1, start_py:end_py] = 1
    
    # Horizontal scans - MATLAB: for k=1:numrows, Ghz(k,k:numrows:numrows*numcols)=1; end
    for k in range(1, numrows + 1):
        # MATLAB k:numrows:numrows*numcols means start at k, step by numrows
        start_py = k - 1  # Convert k to 0-based
        Ghz[k-1, start_py::numrows] = 1
    
    # Diagonal scan 1 - MATLAB: for k=1:numcols, Gdiag1(k,numrows*(k-1)+1:(-numrows+1):k)=1; end
    for k in range(1, numcols + 1):
        start_matlab = numrows * (k-1) + 1  # MATLAB 1-based
        end_matlab = k                      # MATLAB inclusive
        step = -numrows + 1
        
        start_py = start_matlab - 1         # Convert to 0-based
        end_py = end_matlab - 1             # Convert to 0-based
        
        # Generate indices for MATLAB-style range with step
        if start_py >= end_py:  # Valid descending range
            current = start_py
            while current >= end_py:
                if 0 <= current < N:
                    Gdiag1[k-1, current] = 1
                current += step
    
    # Diagonal scan 3 - MATLAB: for k=1:numcols, Gdiag3(k,numrows*k:(numrows-1):numrows*numcols)=1; end
    for k in range(1, numcols + 1):
        start_matlab = numrows * k          # MATLAB 1-based
        end_matlab = numrows * numcols      # MATLAB inclusive
        step = numrows - 1
        
        start_py = start_matlab - 1         # Convert to 0-based
        
        # Generate indices for MATLAB-style range with step
        current = start_py
        while current < N:
            if 0 <= current < N:
                Gdiag3[k-1, current] = 1
            current += step
    
    # Remove first row from Gdiag3 - MATLAB: Gdiag3(1,:)=[];
    Gdiag3 = Gdiag3[1:, :]
    
    # Diagonal scan 2 - MATLAB: for k=1:numcols, Gdiag2(k,numrows*(k-1)+1:(numrows+1):numrows*numcols)=1; end
    for k in range(1, numcols + 1):
        start_matlab = numrows * (k-1) + 1  # MATLAB 1-based
        end_matlab = numrows * numcols      # MATLAB inclusive
        step = numrows + 1
        
        start_py = start_matlab - 1         # Convert to 0-based
        
        # Generate indices for MATLAB-style range with step
        current = start_py
        while current < N:
            if 0 <= current < N:
                Gdiag2[k-1, current] = 1
            current += step
    
    # Diagonal scan 4 - MATLAB: for k=1:numcols, Gdiag4(k,numrows*k:(-numrows-1):1)=1; end
    for k in range(1, numcols + 1):
        start_matlab = numrows * k          # MATLAB 1-based
        end_matlab = 1                      # MATLAB inclusive
        step = -numrows - 1
        
        start_py = start_matlab - 1         # Convert to 0-based
        end_py = end_matlab - 1             # Convert to 0-based (so end at 0)
        
        # Generate indices for MATLAB-style range with step
        current = start_py
        while current >= end_py:
            if 0 <= current < N:
                Gdiag4[k-1, current] = 1
            current += step
    
    # Remove last row from Gdiag4 - MATLAB: Gdiag4(end,:)=[];
    Gdiag4 = Gdiag4[:-1, :]
    
    # Combine diagonal matrices - MATLAB: G1diag=[Gdiag1;Gdiag3]; G2diag=[Gdiag4;Gdiag2];
    G1diag = np.vstack([Gdiag1, Gdiag3])
    G2diag = np.vstack([Gdiag4, Gdiag2])
    
    # Scale by lengths - MATLAB: Ghz=cell_length*Ghz; etc.
    Ghz = cell_length * Ghz
    Gvert = cell_length * Gvert
    G1diag = np.sqrt(2) * cell_length * G1diag
    G2diag = np.sqrt(2) * cell_length * G2diag
    
    return Ghz, Gvert, G1diag, G2diag

# Build G matrices
Ghz, Gvert, G1diag, G2diag = make_scanG(numrows, numcols, cell_length)
